In [2]:
# Only execute once: 
# pip install pennylane_qiskit

## Imports

In [1]:
import pennylane as qml
import pennylane_qiskit as pq

In [3]:
# For example: 

# Let us import circ_1 from circuit1.py
from circuit_2 import circ_2 as qc

# Let us obtain the QISKIT circuit stored inside the circ_1 object
qiskit_circ = qc.get_quantum_circuit()

# To check our understanding, we can print the types of each object (as below)
print(f'The type of circ_1 is {str(type(qc))[7:]}\n')
print(f'After we use the method "get_quantum_circuit()", \
we can obtain the actual circuit, whose type is {str(type(qiskit_circ))[7:]}\n')

# Since we have the QISKIT circuit, we can convert it to a PennyLane circuit
pennylane_circ = pq.load(qiskit_circ)

# We can print the circuit using Pennylane's draw command
# Notice we are using the Pennylane library (qml)
circ_drawer = qml.draw(pennylane_circ)
print(circ_drawer())

The type of circ_1 is 'gate_configurations.circuit'>

After we use the method "get_quantum_circuit()", we can obtain the actual circuit, whose type is 'qiskit.circuit.quantumcircuit.QuantumCircuit'>

0: ──RX(2.22)──RZ(4.23)─╭●───────┤  
1: ──RX(1.44)──RZ(4.85)─╰X─╭●────┤  
2: ──RX(2.78)──RZ(4.50)────╰X─╭●─┤  
3: ──RX(4.58)──RZ(2.97)───────╰X─┤  


In [4]:
qc.get_rotation_params()

{'rx1': 2.216405686489437,
 'rx2': 1.4412926089581348,
 'rx3': 2.775712586615909,
 'rx4': 4.5818375927641934,
 'rz5': 4.227467125540714,
 'rz6': 4.845487852393711,
 'rz7': 4.504397364286557,
 'rz8': 2.968352665748927}

### You must execute the following code to convert a QISKIT circuit into a Pennylane quantum circuit. Look at the example above for more details. 

```
qiskit_circ = qc.get_quantum_circuit()

pennylane_circ = pq.load(qiskit_circ)

# To visualize only
circ_drawer = qml.draw(pennylane_circ)
print(circ_drawer)
```